Reference: https://github.com/dh1105/Sentence-Entailment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install transformers
!pip install sentencepiece
## !pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp36-cp36m-linux_x86_64.whl

In [1]:
import pandas as pd
import re
import torch
# import torch_xla
# import torch_xla.core.xla_model as xm
from torch.utils.data import Dataset, TensorDataset, DataLoader, SequentialSampler, RandomSampler
from torch.nn.utils.rnn import pad_sequence
# from keras.preprocessing.sequence import pad_sequences
import pickle
import os
import numpy as np

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
base_path = '/content/drive/MyDrive/COMPSCI 685 Advanced Natural Language Processing/Project/'
task_a_hyp = "This is offensive speech."
task_b_hyp = "This is targeted offense."
task_c_hyp_1= "This is targeted towards an individual."
task_c_hyp_2 = "This is targeted towards a group."
task_c_hyp_3 = "This isn't targeted towards a group or an individual."
f = open(base_path + 'Data/OffensEval/olid_train_v2.csv')
f_a = open(base_path + 'Data/OffensEval/bert_nli_train.csv', 'w')
f_a.write('tweet_id' + '\t' + 'gold_label' + '\t' + 'sentence1' + '\t' + 'sentence2' + '\n')
lines = f.readlines()
for line in lines[1:]:
  row = line.split('\t')
  tweet_id = row[0].strip()
  tweet_text = ' '.join(row[1:-3]).strip()
  is_offensive = row[-3].strip()
  is_targeted = row[-2].strip()
  target = row[-1].strip()
  if is_offensive == 'OFF':
    f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_a_hyp + '\n')
    if is_targeted == 'TIN':
      f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_b_hyp + '\n')
      if target == 'IND':
        f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_c_hyp_1 + '\n')
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_c_hyp_2 + '\n')
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_c_hyp_3 + '\n')
      elif target == 'GRP':
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_c_hyp_1 + '\n')
        f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_c_hyp_2 + '\n')
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_c_hyp_3 + '\n')
      elif target == 'OTH':
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_c_hyp_1 + '\n')
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_c_hyp_2 + '\n')
        f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_c_hyp_3 + '\n')
    elif is_targeted == 'UNT':
      f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_b_hyp + '\n')
  else:
    f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_a_hyp + '\n')
f.close()
f_a.close()

In [4]:
task_a_hyp = "This is offensive speech."
task_b_hyp = "This is targeted offense."
task_c_hyp_1= "This is targeted towards an individual."
task_c_hyp_2 = "This is targeted towards a group."
task_c_hyp_3 = "This isn't targeted towards a group or an individual."
f = open(base_path + 'Data/OffensEval/olid_val_v2.csv')
f_a = open(base_path + 'Data/OffensEval/bert_nli_val.csv', 'w')
f_a.write('tweet_id' + '\t' + 'gold_label' + '\t' + 'sentence1' + '\t' + 'sentence2' + '\n')
lines = f.readlines()
for line in lines[1:]:
  row = line.split('\t')
  tweet_id = row[0].strip()
  tweet_text = ' '.join(row[1:-3]).strip()
  is_offensive = row[-3].strip()
  is_targeted = row[-2].strip()
  target = row[-1].strip()
  if is_offensive == 'OFF':
    f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_a_hyp + '\n')
    if is_targeted == 'TIN':
      f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_b_hyp + '\n')
      if target == 'IND':
        f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_c_hyp_1 + '\n')
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_c_hyp_2 + '\n')
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_c_hyp_3 + '\n')
      elif target == 'GRP':
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_c_hyp_1 + '\n')
        f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_c_hyp_2 + '\n')
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_c_hyp_3 + '\n')
      elif target == 'OTH':
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_c_hyp_1 + '\n')
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_c_hyp_2 + '\n')
        f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_c_hyp_3 + '\n')
    elif is_targeted == 'UNT':
      f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_b_hyp + '\n')
  else:
    f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_a_hyp + '\n')
f.close()
f_a.close()

In [75]:
hyp = "This is offensive speech."
f = open(base_path + 'Data/OffensEval/olid_test_subtask_a.csv')
f_a = open(base_path + 'Data/OffensEval/bert_nli_test_a.csv', 'w')
f_a.write('tweet_id' + '\t' + 'gold_label' + '\t' + 'sentence1' + '\t' + 'sentence2' + '\n')
for line in f.readlines()[1:]:
    tweet_id, tweet_text, label = [x.strip() for x in line.split('\t')]
    if label == 'OFF':
        f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + hyp + '\n')
    else:
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + hyp + '\n')
f_a.close()
f.close()
hyp = "This is targeted offense."
f = open(base_path + 'Data/OffensEval/olid_test_subtask_b.csv')
f_a = open(base_path + 'Data/OffensEval/bert_nli_test_b.csv', 'w')
f_a.write('tweet_id' + '\t' + 'gold_label' + '\t' + 'sentence1' + '\t' + 'sentence2' + '\n')
for line in f.readlines()[1:]:
    tweet_id, tweet_text, label = [x.strip() for x in line.split('\t')]
    if label == 'TIN':
        f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + hyp + '\n')
    else:
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + hyp + '\n')
f_a.close()
f.close()
hyp_1= "This is targeted towards an individual."
hyp_2 = "This is targeted towards a group."
hyp_3 = "This isn't targeted towards a group or an individual."
f = open(base_path + 'Data/OffensEval/olid_test_subtask_c.csv')
f_a = open(base_path + 'Data/OffensEval/bert_nli_test_c.csv', 'w')
f_a.write('tweet_id' + '\t' + 'gold_label' + '\t' + 'sentence1' + '\t' + 'sentence2' + '\n')
for line in f.readlines()[1:]:
    tweet_id, tweet_text, label = [x.strip() for x in line.split('\t')]
    if label == 'IND':
        f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + hyp_1 + '\n')
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + hyp_2 + '\n')
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + hyp_3 + '\n')
    elif label == 'GRP':
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + hyp_1 + '\n')
        f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + hyp_2 + '\n')
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + hyp_3 + '\n')
    else:
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + hyp_1 + '\n')
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + hyp_2 + '\n')
        f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + hyp_3 + '\n')
f_a.close()
f.close()

In [76]:
test_df_a = pd.read_csv(base_path + 'Data/OffensEval/bert_nli_test_a.csv', delimiter='\t')
test_df_b = pd.read_csv(base_path + 'Data/OffensEval/bert_nli_test_b.csv', delimiter='\t')
test_df_c = pd.read_csv(base_path + 'Data/OffensEval/bert_nli_test_c.csv', delimiter='\t')
train_df = pd.read_csv(base_path + 'Data/OffensEval/bert_nli_train.csv', delimiter='\t')
val_df = pd.read_csv(base_path + 'Data/OffensEval/bert_nli_val.csv', delimiter='\t')

In [66]:
train_df = train_df.dropna()
val_df = val_df.dropna()
test_df_a = test_df_a.dropna()
test_df_b = test_df_b.dropna()
test_df_c = test_df_c.dropna()

In [8]:
train_df['sentence1'] = train_df['sentence1'].astype(str)
train_df['sentence2'] = train_df['sentence2'].astype(str)

In [9]:
val_df['sentence1'] = val_df['sentence1'].astype(str)
val_df['sentence2'] = val_df['sentence2'].astype(str)

In [10]:
test_df_a['sentence1'] = test_df_a['sentence1'].astype(str)
test_df_a['sentence2'] = test_df_a['sentence2'].astype(str)
test_df_b['sentence1'] = test_df_b['sentence1'].astype(str)
test_df_b['sentence2'] = test_df_b['sentence2'].astype(str)
test_df_c['sentence1'] = test_df_c['sentence1'].astype(str)
test_df_c['sentence2'] = test_df_c['sentence2'].astype(str)

In [11]:
train_df = train_df[(train_df['sentence1'].str.split().str.len() > 0) & (train_df['sentence2'].str.split().str.len() > 0)]
val_df = val_df[(val_df['sentence1'].str.split().str.len() > 0) & (val_df['sentence2'].str.split().str.len() > 0)]
test_df_a = test_df_a[(test_df_a['sentence1'].str.split().str.len() > 0) & (test_df_a['sentence2'].str.split().str.len() > 0)]
test_df_b = test_df_b[(test_df_b['sentence1'].str.split().str.len() > 0) & (test_df_b['sentence2'].str.split().str.len() > 0)]
test_df_c = test_df_c[(test_df_c['sentence1'].str.split().str.len() > 0) & (test_df_c['sentence2'].str.split().str.len() > 0)]

In [12]:
test_df_a = test_df_a.sort_values(by=['tweet_id'])
test_df_b = test_df_b.sort_values(by=['tweet_id'])
temp = test_df_c.copy()
test_df_c = temp[temp.sentence2 == "This is targeted towards an individual."].sort_values(by=['tweet_id'])
test_df_d = temp[temp.sentence2 == "This is targeted towards a group."].sort_values(by=['tweet_id'])
test_df_e = temp[temp.sentence2 == "This isn't targeted towards a group or an individual."].sort_values(by=['tweet_id'])

In [13]:
print(len(test_df_a))
print(len(test_df_b))
print(len(test_df_c))
print(len(test_df_d))
print(len(test_df_e))

860
860
213
213
213


In [77]:
test_df = pd.concat([test_df_a, test_df_b, test_df_c, test_df_d, test_df_e], axis=0, ignore_index=True) 

In [15]:
import torch
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import pickle
import os
from transformers import BertTokenizer

class MNLIDataBert(Dataset):

  def __init__(self, train_df, val_df, test_df, test_df_a, test_df_b, test_df_c, test_df_d, test_df_e):
    self.label_dict = {'entailment': 1, 'contradiction': 0}

    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df
    self.test_df_a = test_df_a
    self.test_df_b = test_df_b
    self.test_df_c = test_df_c
    self.test_df_d = test_df_d
    self.test_df_e = test_df_e
    self.base_path = base_path + 'mnli-data-offenseval'
    self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
    self.train_data = None
    self.val_data = None
    self.test_data = None
    self.test_data_a = None
    self.test_data_b = None
    self.test_data_c = None
    self.test_data_d = None
    self.test_data_e = None
    self.init_data()

  def init_data(self):
    #Saving takes too much RAM
    
    if os.path.exists(os.path.join(self.base_path, 'train_data.pkl')):
      print("Found training data")
      with open(os.path.join(self.base_path, 'train_data.pkl'), 'rb') as f:
        self.train_data = pickle.load(f)
    else:
      self.train_data = self.load_data(self.train_df)
      with open(os.path.join(self.base_path, 'train_data.pkl'), 'wb') as f:
        pickle.dump(self.train_data, f)
    if os.path.exists(os.path.join(self.base_path, 'val_data.pkl')):
      print("Found val data")
      with open(os.path.join(self.base_path, 'val_data.pkl'), 'rb') as f:
        self.val_data = pickle.load(f)
    else:
      self.val_data = self.load_data(self.val_df)
      with open(os.path.join(self.base_path, 'val_data.pkl'), 'wb') as f:
        pickle.dump(self.val_data, f)
    if os.path.exists(os.path.join(self.base_path, 'test_data.pkl')):
      print("Found test data")
      with open(os.path.join(self.base_path, 'test_data.pkl'), 'rb') as f:
        self.test_data = pickle.load(f)
    else:
      self.test_data = self.load_data(self.test_df)
      with open(os.path.join(self.base_path, 'test_data.pkl'), 'wb') as f:
        pickle.dump(self.test_data, f)
    if os.path.exists(os.path.join(self.base_path, 'test_data_a.pkl')):
      print("Found test data a")
      with open(os.path.join(self.base_path, 'test_data_a.pkl'), 'rb') as f:
        self.test_data_a = pickle.load(f)
    else:
      self.test_data_a = self.load_data(self.test_df_a)
      with open(os.path.join(self.base_path, 'test_data_a.pkl'), 'wb') as f:
        pickle.dump(self.test_data_a, f)
    if os.path.exists(os.path.join(self.base_path, 'test_data_b.pkl')):
      print("Found test data b")
      with open(os.path.join(self.base_path, 'test_data_b.pkl'), 'rb') as f:
        self.test_data_b = pickle.load(f)
    else:
      self.test_data_b = self.load_data(self.test_df_b)
      with open(os.path.join(self.base_path, 'test_data_b.pkl'), 'wb') as f:
        pickle.dump(self.test_data_b, f)
    if os.path.exists(os.path.join(self.base_path, 'test_data_c.pkl')):
      print("Found test data c")
      with open(os.path.join(self.base_path, 'test_data_c.pkl'), 'rb') as f:
        self.test_data_c = pickle.load(f)
    else:
      self.test_data_c = self.load_data(self.test_df_c)
      with open(os.path.join(self.base_path, 'test_data_c.pkl'), 'wb') as f:
        pickle.dump(self.test_data_c, f)
    if os.path.exists(os.path.join(self.base_path, 'test_data_d.pkl')):
      print("Found test data d")
      with open(os.path.join(self.base_path, 'test_data_d.pkl'), 'rb') as f:
        self.test_data_d = pickle.load(f)
    else:
      self.test_data_d = self.load_data(self.test_df_d)
      with open(os.path.join(self.base_path, 'test_data_d.pkl'), 'wb') as f:
        pickle.dump(self.test_data_d, f)
    if os.path.exists(os.path.join(self.base_path, 'test_data_e.pkl')):
      print("Found test data e")
      with open(os.path.join(self.base_path, 'test_data_e.pkl'), 'rb') as f:
        self.test_data_e = pickle.load(f)
    else:
      self.test_data_e = self.load_data(self.test_df_e)
      with open(os.path.join(self.base_path, 'test_data_e.pkl'), 'wb') as f:
        pickle.dump(self.test_data_e, f)
    # self.train_data = self.load_data(self.train_df)
    # self.val_data = self.load_data(self.val_df)
    # self.test_data = self.load_data(self.test_df)
  def load_data(self, df):
    MAX_LEN = 512
    token_ids = []
    mask_ids = []
    seg_ids = []
    y = []

    premise_list = df['sentence1'].to_list()
    hypothesis_list = df['sentence2'].to_list()
    label_list = df['gold_label'].to_list()

    for (premise, hypothesis, label) in zip(premise_list, hypothesis_list, label_list):
      premise_id = self.tokenizer.encode(premise, add_special_tokens = False)
      hypothesis_id = self.tokenizer.encode(hypothesis, add_special_tokens = False)
      pair_token_ids = [self.tokenizer.cls_token_id] + premise_id + [self.tokenizer.sep_token_id] + hypothesis_id + [self.tokenizer.sep_token_id]
      premise_len = len(premise_id)
      hypothesis_len = len(hypothesis_id)

      segment_ids = torch.tensor([0] * (premise_len + 2) + [1] * (hypothesis_len + 1))  # sentence 0 and sentence 1
      attention_mask_ids = torch.tensor([1] * (premise_len + hypothesis_len + 3))  # mask padded values

      token_ids.append(torch.tensor(pair_token_ids))
      seg_ids.append(segment_ids)
      mask_ids.append(attention_mask_ids)
      y.append(self.label_dict[label])
    
    token_ids = pad_sequence(token_ids, batch_first=True)
    mask_ids = pad_sequence(mask_ids, batch_first=True)
    seg_ids = pad_sequence(seg_ids, batch_first=True)
    y = torch.tensor(y)
    dataset = TensorDataset(token_ids, mask_ids, seg_ids, y)
    print(len(dataset))
    return dataset

  def get_data_loaders(self, batch_size=32, shuffle=True):
    train_loader = DataLoader(
      self.train_data,
      shuffle=shuffle,
      batch_size=batch_size
    )

    val_loader = DataLoader(
      self.val_data,
      shuffle=shuffle,
      batch_size=batch_size
    )

    test_loader = DataLoader(
      self.test_data,
      shuffle=shuffle,
      batch_size=batch_size
    )

    test_loader_a = DataLoader(
      self.test_data_a,
      shuffle=shuffle,
      batch_size=batch_size
    )

    test_loader_b = DataLoader(
      self.test_data_b,
      shuffle=shuffle,
      batch_size=batch_size
    )

    test_loader_c = DataLoader(
      self.test_data_c,
      shuffle=shuffle,
      batch_size=batch_size
    )

    test_loader_d = DataLoader(
      self.test_data_d,
      shuffle=shuffle,
      batch_size=batch_size
    )

    test_loader_e = DataLoader(
      self.test_data_e,
      shuffle=shuffle,
      batch_size=batch_size
    )

    return train_loader, val_loader, test_loader, test_loader_a, test_loader_b, test_loader_c, test_loader_d, test_loader_e

In [80]:
mnli_dataset = MNLIDataBert(train_df, val_df, test_df, test_df_a, test_df_b, test_df_c, test_df_d, test_df_e)

29268
1914
2165
860
240
639
213
213


In [81]:
train_loader, val_loader, test_loader, test_loader_a, test_loader_b, test_loader_c, test_loader_d, test_loader_e = mnli_dataset.get_data_loaders(batch_size=64)

In [18]:
from transformers import BertForSequenceClassification, AdamW

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [19]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [20]:
# This variable contains all of the hyperparemeter information our training loop needs
optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, correct_bias=False)

In [21]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 109,483,778 trainable parameters


In [22]:
def multi_acc(y_pred, y_test):
  acc = (torch.log_softmax(y_pred, dim=1).argmax(dim=1) == y_test).sum().float() / float(y_test.size(0))
  return acc

In [23]:
import time

EPOCHS = 5

def train(model, train_loader, val_loader, optimizer):  
  total_step = len(train_loader)
  for epoch in range(EPOCHS):
    start = time.time()
    model.train()
    total_train_loss = 0
    total_train_acc  = 0
    for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(train_loader):
      optimizer.zero_grad()
      if batch_idx%100 == 0:
          print(batch_idx, len(train_loader))
      pair_token_ids = pair_token_ids.to(device)
      mask_ids = mask_ids.to(device)
      seg_ids = seg_ids.to(device)
      labels = y.to(device)
      # prediction = model(pair_token_ids, mask_ids, seg_ids)
      loss, prediction = model(pair_token_ids, 
                             token_type_ids=seg_ids, 
                             attention_mask=mask_ids, 
                             labels=labels).values()

      # loss = criterion(prediction, labels)
      acc = multi_acc(prediction, labels)

      loss.backward()
      optimizer.step()
      
      total_train_loss += loss.item()
      total_train_acc  += acc.item()

    train_acc  = total_train_acc/len(train_loader)
    train_loss = total_train_loss/len(train_loader)
    model.eval()
    total_val_acc  = 0
    total_val_loss = 0
    with torch.no_grad():
      for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(val_loader):
        optimizer.zero_grad()
        pair_token_ids = pair_token_ids.to(device)
        mask_ids = mask_ids.to(device)
        seg_ids = seg_ids.to(device)
        labels = y.to(device)

        # prediction = model(pair_token_ids, mask_ids, seg_ids)
        loss, prediction = model(pair_token_ids, 
                             token_type_ids=seg_ids, 
                             attention_mask=mask_ids, 
                             labels=labels).values()
        
        # loss = criterion(prediction, labels)
        acc = multi_acc(prediction, labels)

        total_val_loss += loss.item()
        total_val_acc  += acc.item()

    val_acc  = total_val_acc/len(val_loader)
    val_loss = total_val_loss/len(val_loader)
    end = time.time()
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)
    torch.save(model.state_dict(),os.path.join(base_path + 'WeightsOffenseval', 'nli_{}.pt'.format(epoch)))
    print(f'Epoch {epoch+1}: train_loss: {train_loss:.4f} train_acc: {train_acc:.4f} | val_loss: {val_loss:.4f} val_acc: {val_acc:.4f}')
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

In [25]:
import time
import math
def test(model, test_loader):
  model.load_state_dict(torch.load(base_path+'WeightsOffenseval/nli_3.pt', map_location=torch.device('cpu')))
  model.eval()
  total_test_acc  = 0
  total_test_loss = 0
  start = time.time()
  preds = []
  acts = []
  with torch.no_grad():
    #print("Prediction", "Label")
    for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(test_loader):
      optimizer.zero_grad()
      pair_token_ids = pair_token_ids.to(device)
      mask_ids = mask_ids.to(device)
      seg_ids = seg_ids.to(device)
      labels = y.to(device)

      # prediction = model(pair_token_ids, mask_ids, seg_ids)
      loss, prediction = model(pair_token_ids, 
                           token_type_ids=seg_ids, 
                           attention_mask=mask_ids, 
                           labels=labels).values()
        
      # loss = criterion(prediction, labels)
      acc = multi_acc(prediction, labels)
      preds = preds + prediction.tolist()
      acts = acts + labels.tolist()
      #print(prediction, labels)
      total_test_loss += loss.item()
      total_test_acc  += acc.item() 

  test_acc  = total_test_acc/len(test_loader)
  test_loss = total_test_loss/len(test_loader)
  end = time.time()
  hours, rem = divmod(end-start, 3600)
  minutes, seconds = divmod(rem, 60)
  print(f'test_loss: {test_loss:.4f} test_acc: {test_acc:.4f}')
  print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
  #return [math.exp(preds[i][1])/(math.exp(preds[i][0] + math.exp(preds[i][1]))) for i in range(len(preds))], acts
  #return [0 if preds[i][0] > preds[i][1] else 1 for i in range(len(preds))], acts
  return preds, acts

In [24]:
train(model, train_loader, val_loader, optimizer)

0 458
100 458
200 458
300 458
400 458
Epoch 1: train_loss: 0.2690 train_acc: 0.8965 | val_loss: 0.1696 val_acc: 0.9390
00:12:51.07
0 458


KeyboardInterrupt: 

In [82]:
preds, acts = test(model, test_loader)

test_loss: 0.4694 test_acc: 0.7990
00:00:11.02


In [27]:
preds_a, acts_a = test(model, test_loader_a)

test_loss: 0.4511 test_acc: 0.8139
00:00:04.18


In [28]:
#CLASS A:
tp_a = 0
fp_a = 0
tn_a = 0
fn_a = 0
preds_a = [math.exp(preds_a[i][1])/(math.exp(preds_a[i][0]) + math.exp(preds_a[i][1])) for i in range(len(preds_a))]
for i in range(len(preds_a)):
    if acts_a[i] == 1:
        if preds_a[i] >= 0.5:
            tp_a += 1
        else:
            fn_a += 1
    else:
        if preds_a[i] < 0.5:
            tn_a += 1
        else:
            fp_a += 1
accuracy_a = (tp_a + tn_a)/(tp_a + fp_a + fn_a + tn_a)
recall_a = tp_a/(tp_a + fn_a)
precision_a = tp_a/(tp_a + fp_a)
f1_a = (2*precision_a*recall_a)/(precision_a + recall_a)
print("TP: {}, FP: {}\nFN: {}, TN: {}".format(tp_a, fp_a, fn_a, tn_a))
print("Recall:", recall_a)
print("Precsion:", precision_a)
print("F1:", f1_a)
print("Accuracy", accuracy_a)

TP: 189, FP: 108
FN: 51, TN: 512
Recall: 0.7875
Precsion: 0.6363636363636364
F1: 0.7039106145251396
Accuracy 0.8151162790697675


In [29]:
ent_acts_a = 0
cont_acts_a = 0
correct_ents_a = 0
correct_conts_a = 0
for i in range(len(preds_a)):
    if acts_a[i] == 1:
        ent_acts_a += 1
        if preds_a[i] >= 0.5:
            correct_ents_a += 1
    else:
        cont_acts_a += 1
        if preds_a[i] < 0.5:
            correct_conts_a += 1
            
print("Total offensive examples:", ent_acts_a)
print("Correctly predicted offensive examples:", correct_ents_a)
print("Accuracy for predicting offensive examples:", correct_ents_a/ent_acts_a)
print("Total non offensive examples:", cont_acts_a)
print("Correctly predicted non offensive examples:", correct_conts_a)
print("Accuracy for predicting non offensive examples:", correct_conts_a/cont_acts_a)

Total offensive examples: 240
Correctly predicted offensive examples: 189
Accuracy for predicting offensive examples: 0.7875
Total non offensive examples: 620
Correctly predicted non offensive examples: 512
Accuracy for predicting non offensive examples: 0.8258064516129032


In [83]:
preds_b, acts_b = test(model, test_loader_b)

test_loss: 0.2992 test_acc: 0.8854
00:00:01.16


In [84]:
#CLASS B:
tp_b = 0
fp_b = 0
tn_b = 0
fn_b = 0
preds_b = [math.exp(preds_b[i][1])/(math.exp(preds_b[i][0]) + math.exp(preds_b[i][1])) for i in range(len(preds_b))]
for i in range(len(preds_b)):
    if acts_b[i] == 1:
        if preds_b[i] >= 0.5:
            tp_b += 1
        else:
            fn_b += 1
    else:
        if preds_b[i] < 0.5:
            tn_b += 1
        else:
            fp_b += 1
accuracy_b = (tp_b + tn_b)/(tp_b + fp_b + fn_b + tn_b)
recall_b = tp_b/(tp_b + fn_b)
precision_b = tp_b/(tp_b + fp_b)
f1_b = (2*precision_b*recall_b)/(precision_b + recall_b)
print("TP: {}, FP: {}\nFN: {}, TN: {}".format(tp_b, fp_b, fn_b, tn_b))
print("Recall:", recall_b)
print("Precsion:", precision_b)
print("F1:", f1_b)
print("Accuracy", accuracy_b)

TP: 203, FP: 17
FN: 10, TN: 10
Recall: 0.9530516431924883
Precsion: 0.9227272727272727
F1: 0.9376443418013857
Accuracy 0.8875


In [32]:
ent_acts_b = 0
cont_acts_b = 0
correct_ents_b = 0
correct_conts_b = 0
for i in range(len(preds_b)):
    if acts_b[i] == 1:
        ent_acts_b += 1
        if preds_b[i] > 0.5:
            correct_ents_b += 1
    else:
        cont_acts_b += 1
        if preds_b[i] < 0.5:
            correct_conts_b += 1
            
print("Total targeted offensive examples:", ent_acts_b)
print("Correctly predicted targeted offensive examples:", correct_ents_b)
print("Accuracy for predicting targeted offensive examples:", correct_ents_b/ent_acts_b)
print("Total non targeted offensive examples:", cont_acts_b)
print("Correctly predicted non targeted offensive examples:", correct_conts_b)
print("Accuracy for predicting non targeted offensive examples:", correct_conts_b/cont_acts_b)

Total targeted offensive examples: 240
Correctly predicted targeted offensive examples: 189
Accuracy for predicting targeted offensive examples: 0.7875
Total non targeted offensive examples: 620
Correctly predicted non targeted offensive examples: 512
Accuracy for predicting non targeted offensive examples: 0.8258064516129032


In [33]:
preds_c, acts_c = test(model, test_loader_c)

test_loss: 0.5668 test_acc: 0.8080
00:00:01.07


In [34]:
#CLASS C:
tp_c = 0
fp_c = 0
tn_c = 0
fn_c = 0
preds_c = [math.exp(preds_c[i][1])/(math.exp(preds_c[i][0]) + math.exp(preds_c[i][1])) for i in range(len(preds_c))]
for i in range(len(preds_c)):
    if acts_c[i] == 1:
        if preds_c[i] >= 0.5:
            tp_c += 1
        else:
            fn_c += 1
    else:
        if preds_c[i] < 0.5:
            tn_c += 1
        else:
            fp_c += 1
accuracy_c = (tp_c + tn_c)/(tp_c + fp_c + fn_c + tn_c)
recall_c = tp_c/(tp_c + fn_c)
precision_c = tp_c/(tp_c + fp_c)
f1_c = (2*precision_c*recall_c)/(precision_c + recall_c)
print("TP: {}, FP: {}\nFN: {}, TN: {}".format(tp_c, fp_c, fn_c, tn_c))
print("Recall:", recall_c)
print("Precsion:", precision_c)
print("F1:", f1_c)
print("Accuracy", accuracy_c)

TP: 79, FP: 22
FN: 21, TN: 91
Recall: 0.79
Precsion: 0.7821782178217822
F1: 0.7860696517412936
Accuracy 0.7981220657276995


In [35]:
ent_acts_c = 0
cont_acts_c = 0
correct_ents_c = 0
correct_conts_c = 0
for i in range(len(preds_c)):
    if acts_c[i] == 1:
        ent_acts_c += 1
        if preds_c[i] >= 0.5:
            correct_ents_c += 1
    else:
        cont_acts_c += 1
        if preds_c[i] < 1:
            correct_conts_c += 1
            
print("Total individual targeted offensive examples:", ent_acts_c)
print("Correctly predicted individual targeted offensive examples:", correct_ents_c)
print("Accuracy for predicting individual targeted offensive examples:", correct_ents_c/ent_acts_c)
print("Total non individual targeted offensive examples:", cont_acts_c)
print("Correctly predicted non individual targeted offensive examples:", correct_conts_c)
print("Accuracy for predicting non individual targeted offensive examples:", correct_conts_c/cont_acts_c)

Total individual targeted offensive examples: 100
Correctly predicted individual targeted offensive examples: 79
Accuracy for predicting individual targeted offensive examples: 0.79
Total non individual targeted offensive examples: 113
Correctly predicted non individual targeted offensive examples: 113
Accuracy for predicting non individual targeted offensive examples: 1.0


In [36]:
preds_d, acts_d = test(model, test_loader_d)

test_loss: 0.5722 test_acc: 0.7329
00:00:01.09


In [37]:
#CLASS D:
tp_d = 0
fp_d = 0
tn_d = 0
fn_d = 0
preds_d = [math.exp(preds_d[i][1])/(math.exp(preds_d[i][0]) + math.exp(preds_d[i][1])) for i in range(len(preds_d))]
for i in range(len(preds_d)):
    if acts_d[i] == 1:
        if preds_d[i] >= 0.5:
            tp_d += 1
        else:
            fn_d += 1
    else:
        if preds_d[i] < 0.5:
            tn_d += 1
        else:
            fp_d += 1
accuracy_d = (tp_d + tn_d)/(tp_d + fp_d + fn_d + tn_d)
recall_d = tp_d/(tp_d + fn_d)
precision_d = tp_d/(tp_d + fp_d)
f1_d = (2*precision_d*recall_d)/(precision_d + recall_d)
print("TP: {}, FP: {}\nFN: {}, TN: {}".format(tp_d, fp_d, fn_d, tn_d))
print("Recall:", recall_d)
print("Precsion:", precision_d)
print("F1:", f1_d)
print("Accuracy", accuracy_d)

TP: 49, FP: 23
FN: 29, TN: 112
Recall: 0.6282051282051282
Precsion: 0.6805555555555556
F1: 0.6533333333333333
Accuracy 0.755868544600939


In [38]:
ent_acts_d = 0
cont_acts_d = 0
correct_ents_d = 0
correct_conts_d = 0
for i in range(len(preds_d)):
    if acts_d[i] == 1:
        ent_acts_d += 1
        if preds_d[i] >= 0.5:
            correct_ents_d += 1
    else:
        cont_acts_d += 1
        if preds_d[i] < 0.5:
            correct_conts_d += 1
            
print("Total group targeted offensive examples:", ent_acts_d)
print("Correctly predicted group targeted offensive examples:", correct_ents_d)
print("Accuracy for predicting group targeted offensive examples:", correct_ents_d/ent_acts_d)
print("Total non group targeted offensive examples:", cont_acts_d)
print("Correctly predicted non group targeted offensive examples:", correct_conts_d)
print("Accuracy for predicting non group targeted offensive examples:", correct_conts_d/cont_acts_d)

Total group targeted offensive examples: 78
Correctly predicted group targeted offensive examples: 49
Accuracy for predicting group targeted offensive examples: 0.6282051282051282
Total non group targeted offensive examples: 135
Correctly predicted non group targeted offensive examples: 112
Accuracy for predicting non group targeted offensive examples: 0.8296296296296296


In [39]:
preds_e, acts_e = test(model, test_loader_e)

test_loss: 0.4725 test_acc: 0.7608
00:00:01.09


In [40]:
#CLASS E:
tp_e = 0
fp_e = 0
tn_e = 0
fn_e = 0
preds_e = [math.exp(preds_e[i][1])/(math.exp(preds_e[i][0]) + math.exp(preds_e[i][1])) for i in range(len(preds_e))]
for i in range(len(preds_e)):
    if acts_e[i] == 1:
        if preds_e[i] >= 0.5:
            tp_e += 1
        else:
            fn_e += 1
    else:
        if preds_e[i] < 0.5:
            tn_e += 1
        else:
            fp_e += 1
accuracy_e = (tp_e + tn_e)/(tp_e + fp_e + fn_e + tn_e)
recall_e = tp_e/(tp_e + fn_e)
precision_e = tp_e/(tp_e + fp_e)
f1_e = (2*precision_e*recall_e)/(precision_e + recall_e)
print("TP: {}, FP: {}\nFN: {}, TN: {}".format(tp_e, fp_e, fn_e, tn_e))
print("Recall:", recall_e)
print("Precsion:", precision_e)
print("F1:", f1_e)
print("Accuracy", accuracy_e)

TP: 8, FP: 24
FN: 27, TN: 154
Recall: 0.22857142857142856
Precsion: 0.25
F1: 0.23880597014925375
Accuracy 0.7605633802816901


In [41]:
ent_acts_e = 0
cont_acts_e = 0
correct_ents_e = 0
correct_conts_e = 0
for i in range(len(preds_e)):
    if acts_e[i] == 1:
        ent_acts_e += 1
        if preds_e[i] >= 0.5:
            correct_ents_e += 1
    else:
        cont_acts_e += 1
        if preds_e[i] < 0.5:
            correct_conts_e += 1
            
print("Total other targeted offensive examples:", ent_acts_e)
print("Correctly predicted other targeted offensive examples:", correct_ents_e)
print("Accuracy for predicting other targeted offensive examples:", correct_ents_e/ent_acts_e)
print("Total non other targeted offensive examples:", cont_acts_e)
print("Correctly predicted non other targeted offensive examples:", correct_conts_e)
print("Accuracy for predicting non other targeted offensive examples:", correct_conts_e/cont_acts_e)

Total other targeted offensive examples: 35
Correctly predicted other targeted offensive examples: 8
Accuracy for predicting other targeted offensive examples: 0.22857142857142856
Total non other targeted offensive examples: 178
Correctly predicted non other targeted offensive examples: 154
Accuracy for predicting non other targeted offensive examples: 0.8651685393258427


In [87]:
f1_3 = (f1_c + f1_d + f1_e)/3
print("Macro average F1 for class 3 IND/GRP/OTH is", f1_3)
print("Macro average F1 overall", (f1_a + f1_b + f1_3)/3)

Macro average F1 for class 3 IND/GRP/OTH is 0.5594029850746269
Macro average F1 overall 0.7336526471337175


In [43]:
def test_transfer(model, test_loader):
  model.load_state_dict(torch.load(base_path + 'WeightsHateval/nli_new_4.pt', map_location=torch.device('cpu')))
  model.eval()
  total_test_acc  = 0
  total_test_loss = 0
  start = time.time()
  preds = []
  acts = []
  with torch.no_grad():
    #print("Prediction", "Label")
    for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(test_loader):
      optimizer.zero_grad()
      pair_token_ids = pair_token_ids.to(device)
      mask_ids = mask_ids.to(device)
      seg_ids = seg_ids.to(device)
      labels = y.to(device)

      # prediction = model(pair_token_ids, mask_ids, seg_ids)
      loss, prediction = model(pair_token_ids, 
                           token_type_ids=seg_ids, 
                           attention_mask=mask_ids, 
                           labels=labels).values()
        
      # loss = criterion(prediction, labels)
      acc = multi_acc(prediction, labels)
      preds = preds + prediction.tolist()
      acts = acts + labels.tolist()
      #print(prediction, labels)
      total_test_loss += loss.item()
      total_test_acc  += acc.item() 

  test_acc  = total_test_acc/len(test_loader)
  test_loss = total_test_loss/len(test_loader)
  end = time.time()
  hours, rem = divmod(end-start, 3600)
  minutes, seconds = divmod(rem, 60)
  print(f'test_loss: {test_loss:.4f} test_acc: {test_acc:.4f}')
  print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
  return preds, acts

In [44]:
preds_a_transfer, acts_a_transfer = test_transfer(model, test_loader_a)

test_loss: 1.5004 test_acc: 0.6481
00:00:04.15


In [85]:
preds_b_transfer, acts_b_transfer = test_transfer(model, test_loader_b)

test_loss: 3.1208 test_acc: 0.3333
00:00:01.17


In [46]:
preds_c_transfer, acts_c_transfer = test_transfer(model, test_loader_c)

test_loss: 1.2519 test_acc: 0.6665
00:00:01.06


In [47]:
preds_d_transfer, acts_d_transfer = test_transfer(model, test_loader_d)

test_loss: 2.4865 test_acc: 0.4115
00:00:01.06


In [48]:
preds_e_transfer, acts_e_transfer = test_transfer(model, test_loader_e)

test_loss: 1.5778 test_acc: 0.5917
00:00:01.10


In [49]:
#CLASS A: Transfer learning from hateval

tp_a_transfer = 0
fp_a_transfer = 0
tn_a_transfer = 0
fn_a_transfer = 0
preds_a_transfer = [math.exp(preds_a_transfer[i][1])/(math.exp(preds_a_transfer[i][0]) + math.exp(preds_a_transfer[i][1])) for i in range(len(preds_e))]
for i in range(len(preds_a_transfer)):
    if acts_a_transfer[i] == 1:
        if preds_a_transfer[i] >= 0.5:
            tp_a_transfer += 1
        else:
            fn_a_transfer += 1
    else:
        if preds_a_transfer[i] < 0.5:
            tn_a_transfer += 1
        else:
            fp_a_transfer += 1
accuracy_a_transfer = (tp_a_transfer + tn_a_transfer)/(tp_a_transfer + fp_a_transfer + fn_a_transfer + tn_a_transfer)
recall_a_transfer = tp_a_transfer/(tp_a_transfer + fn_a_transfer)
precision_a_transfer = tp_a_transfer/(tp_a_transfer + fp_a_transfer)
f1_a_transfer = (2*precision_a_transfer*recall_a_transfer)/(precision_a_transfer + recall_a_transfer)
print("TP: {}, FP: {}\nFN: {}, TN: {}".format(tp_a_transfer, fp_a_transfer, fn_a_transfer, tn_a_transfer))
print("Recall:", recall_a_transfer)
print("Precsion:", precision_a_transfer)
print("F1:", f1_a_transfer)
print("Accuracy", accuracy_a_transfer)

TP: 20, FP: 32
FN: 42, TN: 119
Recall: 0.3225806451612903
Precsion: 0.38461538461538464
F1: 0.3508771929824561
Accuracy 0.6525821596244131


In [86]:
#CLASS B: Transfer learning from hateval

tp_b_transfer = 0
fp_b_transfer = 0
tn_b_transfer = 0
fn_b_transfer = 0
preds_b_transfer = [math.exp(preds_b_transfer[i][1])/(math.exp(preds_b_transfer[i][0]) + math.exp(preds_b_transfer[i][1])) for i in range(len(preds_e))]
for i in range(len(preds_b_transfer)):
    if acts_b_transfer[i] == 1:
        if preds_b_transfer[i] >= 0.5:
            tp_b_transfer += 1
        else:
            fn_b_transfer += 1
    else:
        if preds_b_transfer[i] < 0.5:
            tn_b_transfer += 1
        else:
            fp_b_transfer += 1
accuracy_b_transfer = (tp_b_transfer + tn_b_transfer)/(tp_b_transfer + fp_b_transfer + fn_b_transfer + tn_b_transfer)
recall_b_transfer = tp_b_transfer/(tp_b_transfer + fn_b_transfer)
precision_b_transfer = tp_b_transfer/(tp_b_transfer + fp_b_transfer)
f1_b_transfer = (2*precision_b_transfer*recall_b_transfer)/(precision_b_transfer + recall_b_transfer)
print("TP: {}, FP: {}\nFN: {}, TN: {}".format(tp_b_transfer, fp_b_transfer, fn_b_transfer, tn_b_transfer))
print("Recall:", recall_b_transfer)
print("Precsion:", precision_b_transfer)
print("F1:", f1_b_transfer)
print("Accuracy", accuracy_b_transfer)

TP: 55, FP: 9
FN: 136, TN: 13
Recall: 0.2879581151832461
Precsion: 0.859375
F1: 0.4313725490196078
Accuracy 0.3192488262910798


In [51]:
#CLASS C: Transfer learning from hateval

tp_c_transfer = 0
fp_c_transfer = 0
tn_c_transfer = 0
fn_c_transfer = 0
preds_c_transfer = [math.exp(preds_c_transfer[i][1])/(math.exp(preds_c_transfer[i][0]) + math.exp(preds_c_transfer[i][1])) for i in range(len(preds_e))]
for i in range(len(preds_c_transfer)):
    if acts_c_transfer[i] == 1:
        if preds_c_transfer[i] >= 0.5:
            tp_c_transfer += 1
        else:
            fn_c_transfer += 1
    else:
        if preds_c_transfer[i] < 0.5:
            tn_c_transfer += 1
        else:
            fp_c_transfer += 1
accuracy_c_transfer = (tp_c_transfer + tn_c_transfer)/(tp_c_transfer + fp_c_transfer + fn_c_transfer + tn_c_transfer)
recall_c_transfer = tp_c_transfer/(tp_c_transfer + fn_c_transfer)
precision_c_transfer = tp_c_transfer/(tp_c_transfer + fp_c_transfer)
f1_c_transfer = (2*precision_c_transfer*recall_c_transfer)/(precision_c_transfer + recall_c_transfer)
print("TP: {}, FP: {}\nFN: {}, TN: {}".format(tp_c_transfer, fp_c_transfer, fn_c_transfer, tn_c_transfer))
print("Recall:", recall_c_transfer)
print("Precsion:", precision_c_transfer)
print("F1:", f1_c_transfer)
print("Accuracy", accuracy_c_transfer)

TP: 42, FP: 11
FN: 58, TN: 102
Recall: 0.42
Precsion: 0.7924528301886793
F1: 0.5490196078431373
Accuracy 0.676056338028169


In [52]:
#CLASS D: Transfer learning from hateval

tp_d_transfer = 0
fp_d_transfer = 0
tn_d_transfer = 0
fn_d_transfer = 0
preds_d_transfer = [math.exp(preds_d_transfer[i][1])/(math.exp(preds_d_transfer[i][0]) + math.exp(preds_d_transfer[i][1])) for i in range(len(preds_e))]
for i in range(len(preds_d_transfer)):
    if acts_d_transfer[i] == 1:
        if preds_d_transfer[i] >= 0.5:
            tp_d_transfer += 1
        else:
            fn_d_transfer += 1
    else:
        if preds_d_transfer[i] < 0.5:
            tn_d_transfer += 1
        else:
            fp_d_transfer += 1
accuracy_d_transfer = (tp_d_transfer + tn_d_transfer)/(tp_d_transfer + fp_d_transfer + fn_d_transfer + tn_d_transfer)
recall_d_transfer = tp_d_transfer/(tp_d_transfer + fn_d_transfer)
precision_d_transfer = tp_d_transfer/(tp_d_transfer + fp_d_transfer)
f1_d_transfer = (2*precision_d_transfer*recall_d_transfer)/(precision_d_transfer + recall_d_transfer)
print("TP: {}, FP: {}\nFN: {}, TN: {}".format(tp_d_transfer, fp_d_transfer, fn_d_transfer, tn_d_transfer))
print("Recall:", recall_d_transfer)
print("Precsion:", precision_d_transfer)
print("F1:", f1_d_transfer)
print("Accuracy", accuracy_d_transfer)

TP: 6, FP: 50
FN: 72, TN: 85
Recall: 0.07692307692307693
Precsion: 0.10714285714285714
F1: 0.08955223880597014
Accuracy 0.4272300469483568


In [53]:
#CLASS D: Transfer learning from hateval

tp_e_transfer = 0
fp_e_transfer = 0
tn_e_transfer = 0
fn_e_transfer = 0
preds_e_transfer = [math.exp(preds_e_transfer[i][1])/(math.exp(preds_e_transfer[i][0]) + math.exp(preds_e_transfer[i][1])) for i in range(len(preds_e))]
for i in range(len(preds_e_transfer)):
    if acts_e_transfer[i] == 1:
        if preds_e_transfer[i] >= 0.5:
            tp_e_transfer += 1
        else:
            fn_e_transfer += 1
    else:
        if preds_e_transfer[i] < 0.5:
            tn_e_transfer += 1
        else:
            fp_e_transfer += 1
accuracy_e_transfer = (tp_e_transfer + tn_e_transfer)/(tp_e_transfer + fp_e_transfer + fn_e_transfer + tn_e_transfer)
recall_e_transfer = tp_e_transfer/(tp_e_transfer + fn_e_transfer)
precision_e_transfer = tp_e_transfer/(tp_e_transfer + fp_e_transfer)
f1_e_transfer = (2*precision_e_transfer*recall_e_transfer)/(precision_e_transfer + recall_e_transfer)
print("TP: {}, FP: {}\nFN: {}, TN: {}".format(tp_e_transfer, fp_e_transfer, fn_e_transfer, tn_e_transfer))
print("Recall:", recall_e_transfer)
print("Precsion:", precision_e_transfer)
print("F1:", f1_e_transfer)
print("Accuracy", accuracy_e_transfer)

TP: 4, FP: 51
FN: 31, TN: 127
Recall: 0.11428571428571428
Precsion: 0.07272727272727272
F1: 0.08888888888888889
Accuracy 0.6150234741784038


In [54]:
#overall results for transfer learning from hateval

f1_3_transfer = (f1_c_transfer + f1_d_transfer + f1_e_transfer)/3
print("Macro average F1 for class 3 IND/GRP/OTH is", f1_3_transfer)
print("Macro average F1 overall", (f1_a_transfer + f1_b_transfer + f1_3_transfer)/3)

Macro average F1 for class 3 IND/GRP/OTH is 0.2424869118459988
Macro average F1 overall 0.32059505248667797


In [55]:
import time
import math
def test_new(model, test_loader):
  model.load_state_dict(torch.load(base_path + 'WeightsOffenseval/nli_latest_3.pt', map_location=torch.device('cpu')))
  model.eval()
  total_test_acc  = 0
  total_test_loss = 0
  start = time.time()
  preds = []
  acts = []
  wrong_examples = []
  with torch.no_grad():
    #print("Prediction", "Label")
    for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(test_loader):
      optimizer.zero_grad()
      pair_token_ids = pair_token_ids.to(device)
      mask_ids = mask_ids.to(device)
      seg_ids = seg_ids.to(device)
      labels = y.to(device)
      # prediction = model(pair_token_ids, mask_ids, seg_ids)
      loss, prediction = model(pair_token_ids, 
                           token_type_ids=seg_ids, 
                           attention_mask=mask_ids, 
                           labels=labels).values()
        
      # loss = criterion(prediction, labels)
      acc = multi_acc(prediction, labels)
      p = prediction.tolist()
      tmp = [0 if x[0] > x[1] else 1 for x in p]
      for i in range(len(tmp)):
        prediction_made = 1 if p[i][1] > p[i][0] else 0
        actual_answer = labels[i].item()
        if prediction_made != actual_answer:
          l = pair_token_ids[i].tolist()
          s = mnli_dataset.tokenizer.convert_ids_to_tokens(l)
          if '[PAD]' in s:
            wrong_examples.append(' '.join(s[1:s.index('[PAD]')]).replace('##', '') + str(actual_answer))
          else:
            wrong_examples.append(' '.join(s[1:]).replace('##', '') + str(actual_answer))
      preds = preds + prediction.tolist()
      acts = acts + labels.tolist()
      #print(prediction, labels)
      total_test_loss += loss.item()
      total_test_acc  += acc.item() 

  test_acc  = total_test_acc/len(test_loader)
  test_loss = total_test_loss/len(test_loader)
  end = time.time()
  hours, rem = divmod(end-start, 3600)
  minutes, seconds = divmod(rem, 60)
  print(f'test_loss: {test_loss:.4f} test_acc: {test_acc:.4f}')
  print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
  return preds, acts, wrong_examples

In [88]:
p, a, wrong_examples = test_new(model, test_loader)

test_loss: 0.4762 test_acc: 0.8200
00:00:11.11


In [89]:
f = open(base_path + 'Data/OffensEval/wrong_predictions_nli.txt', 'w')
for wrong_example in wrong_examples:
    f.write(wrong_example + '\n')
f.close()

In [90]:
len(wrong_examples)

389